In [11]:
import pandas as pd
import scanpy as sc
import os
import copy
import numpy as np
from scipy import sparse


In [2]:
def maskPerCol(data_train, masked_prob):

    X_train = copy.deepcopy(data_train)
    rows = []
    cols = []
    for col in range(data_train.shape[1]):
        index_pair_train = np.where(data_train[:, col])
        if index_pair_train[0].shape[0] <= 3:
            continue
        masking_idx_train = np.random.choice(index_pair_train[0].shape[0],
                                             int(index_pair_train[0].shape[0] * masked_prob),
                                             replace=False)
        X_train[index_pair_train[0][masking_idx_train], [col] * masking_idx_train.shape[0]] = 0
        for i in index_pair_train[0][masking_idx_train]:
            rows.append(i)
            cols.append(col)

    return X_train, rows, cols

In [3]:
adata = sc.read_h5ad('./processed/data.h5ad')
adata

AnnData object with n_obs × n_vars = 2607 × 13708
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [4]:
sc.pp.normalize_total(adata)
adata.raw = adata

In [5]:
path = './masked'

if not os.path.exists(path):
    os.makedirs(path)

In [6]:
trainData = adata.raw.X.A
trainData.shape

(2607, 13708)

In [7]:
masked, masking_row, masking_col = maskPerCol(trainData, 0.1)

In [9]:
pd.DataFrame(masked, index=adata.obs.index, columns=adata.var.index) \
    .T.to_csv(path + '/data_01.csv')

In [12]:
adata.X = sparse.csr_matrix(masked)
adata.write(path + '/data_01.h5ad')

In [13]:
maskIndex = sparse.coo_matrix(([1] * len(masking_col), (masking_row, masking_col)))

sparse.save_npz(path + '/data_maskIndex_01.csv', maskIndex)

In [8]:
data_train = trainData
masked_prob = 0.1
X_train = copy.deepcopy(data_train)
rows = []
cols = []
i=0
for col in range(data_train.shape[1]):
    i += 1
    NonZeroIndex = np.where(data_train[:, col])
    if NonZeroIndex[0].shape[0]<=3:
        continue

    print("NonZeroIndex")
    print(NonZeroIndex[0])
    print("size: ",NonZeroIndex[0].shape[0])
    print("len: ", int(NonZeroIndex[0].shape[0] * masked_prob))
    print("masking_idx_train")
    
    NonZeroIndexRandomSelection = np.random.choice(NonZeroIndex[0].shape[0],
                                         int(NonZeroIndex[0].shape[0] * masked_prob),
                                         replace=False)
    print(NonZeroIndexRandomSelection)

    print(NonZeroIndex[0][NonZeroIndexRandomSelection])
    print([col] * NonZeroIndexRandomSelection.shape[0])
    print("==================================================")

    
    if i>5:
        break
 

NonZeroIndex
[ 425  989 1076 1130 1319 1391 1572 1739 1753]
size:  9
len:  0
masking_idx_train
[]
[]
[]
NonZeroIndex
[ 102  399  437  787  858 1240 1371 1391 1436 1465 1548 2063 2230 2359
 2530 2556 2569 2594]
size:  18
len:  1
masking_idx_train
[2]
[437]
[4]
NonZeroIndex
[  11   20   33   41   42   55   56   58   61   70  114  120  129  141
  143  158  161  162  169  184  187  194  205  209  216  217  225  234
  252  254  269  275  279  294  295  311  320  321  327  328  338  341
  347  356  380  403  420  426  449  450  456  471  474  518  522  530
  532  535  538  560  574  595  598  602  623  625  632  637  646  647
  648  660  669  689  691  721  737  745  754  758  764  768  805  807
  809  848  853  856  888  900  901  917  927  931  943  945  955  964
  970 1025 1040 1074 1103 1112 1117 1132 1147 1165 1187 1192 1193 1230
 1234 1259 1260 1264 1278 1290 1293 1294 1298 1316 1342 1350 1357 1366
 1369 1375 1378 1389 1406 1426 1437 1445 1453 1460 1465 1466 1475 1482
 1491 1499 1507 1